# Parallelizing your code

In [ ]:
import numpy as np

In [ ]:
import multiprocessing as mp    
mp.set_start_method("fork")    # Only needed for Jupyter, do not do this in python code itself

Often times we want to perform multiple task simulatenous and leverage multiple cores.

Previously we would use the `threading` library or `multiprocessing` library to achieve this but
nowadays these are pretty terrible to use. For multiple reasons but not limited to:
    
    - Script freezing due to deadlock
    - Zombie python processes appearing
    - Script not finishing at the end
    - All other sadness


Instead we will use the `concurrent.futures` module to make this experience as smooth as possible. Lets try it

Python has to parallel execution modules. `threading` and `multiprocessing`. Threading runs parallel jobs in the same python process while Multiprocessing spawns new python processes.

The difference comes from the Global Interpretor Lock. Within a single python process, only one python task can be run at the same time. However if the task release this lock then another task can run. Often time, reading things from file or certain functions (like numba with `nogil`) will release the lock and allow concurrent processes.

Multiproccessing overcomes this by creating a new python process and running tasks there. However it requires explicitly sending data to that process.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
def f(x):
    import time
    time.sleep(1.0)
    return np.sin(x)

with ThreadPoolExecutor(max_workers=1) as e:
    futures = e.submit(f,10)
    if futures.done():
        print('Done')
    else:
        print('Not done')
    print(futures.result())

We can submit a bunch more tasks if we want as well!

In [ ]:
tasks = np.arange(0,10)

with ThreadPoolExecutor(max_workers=8) as e:
    futures = [e.submit(f,x) for x in tasks]
    all_values = [x.result() for x in futures]
    

In [ ]:
all_values

Now this can be a bit laborious for large set of tasks, what we can instead do is leverage the `map` function and map our task across inputs

In [ ]:
%%time
with ThreadPoolExecutor(max_workers=8) as e:
    for x in e.map(f, tasks):
        print(x)

In [ ]:
def g(x):
    import time
    time.sleep(0.3)
    return np.cos(x)

with ThreadPoolExecutor(max_workers=8) as e:
    f_tasks = e.map(f, tasks)
    print('Ok now do g')
    g_tasks = e.map(g, tasks)
    print('Lets see f')
    print(list(f_tasks))
    print('Lets see g')
    print(list(g_tasks))


In [ ]:
import hashlib


def hash_one(n):
    """A somewhat CPU-intensive task."""

    for i in range(1, n):
        hashlib.pbkdf2_hmac("sha256", b"password", b"salt", i * 10000)

    return "done"


def hash_all(n):
    """Function that does hashing in serial."""
    import tqdm
    for i in tqdm.tqdm(range(n)):
        hsh = hash_one(n)

    return "done"

In [ ]:
hash_all(10)

In [ ]:
import tqdm

In [ ]:
def hash_all_process(n):
    """Function that does hashing in serial."""
    import tqdm
    with ProcessPoolExecutor(max_workers=4) as executor:
        for arg, res in tqdm.tqdm(
                            zip(
                                range(n), 
                                executor.map(
                                    hash_one, 
                                    range(n)
                                )
                            ),total=n):
            pass

    return "done"

In [ ]:
hash_all_process(10)